## Библиотеки, загрузка датасетов

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pandas
import pandas

!pip install tqdm
from tqdm import tqdm
tqdm.pandas()
import re
'''
import time
import json
from os import listdir
'''
!pip install pymystem3
from pymystem3 import Mystem
ms = Mystem()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
verse_data = pandas.read_csv('/content/drive/MyDrive/diplom/syllab-tonic-lines.tsv',sep='\t', quotechar='&')

In [ ]:
mono_df_all = pandas.read_csv('/content/drive/MyDrive/diplom/mono_df_all.csv', index_col=0, quotechar='&')

In [ ]:
mono_df_all

,Unnamed: 0,word,ict,line_id,position,clausula,POS_pm,POS_ms,syll_type
0,0,за,False,0,0,Я5ж,PREP,PR,open
1,1,стол,True,0,4,Я5ж,NOUN,S,closed
2,2,я,False,0,6,Я5ж,NPRO,SPRO,open
3,3,вновь,True,0,8,Я5ж,ADVB,ADV,closed
4,4,об,True,1,6,Я5м,PREP,PR,closed
...,...,...,...,...,...,...,...,...,...
3694755,3694755,же,False,2440705,2,Д4д,PRCL,PART,open
3694756,3694756,ты,False,2440705,4,Д4д,NPRO,SPRO,open
3694757,3694757,где,True,2440706,0,Д3м,ADVB,ADVPRO,open
3694758,3694758,же,False,2440706,2,Д3м,PRCL,PART,open


## Сопоставление частей речи

In [ ]:
mono_df_all.groupby('POS_ms')['POS_pm'].unique()

POS_ms
A           [ADJS, ADJF, NOUN, VERB, PRCL, PRTS, nan, GRND]
ADV       [ADVB, PRED, PRCL, PREP, NOUN, nan, INTJ, VERB...
ADVPRO                  [ADVB, CONJ, VERB, NOUN, nan, PRCL]
APRO        [ADJF, NPRO, CONJ, PRCL, ADJS, VERB, nan, NOUN]
COM                                       [nan, ADVB, NOUN]
CONJ                          [CONJ, PRCL, NOUN, ADVB, nan]
INTJ              [PREP, CONJ, INTJ, nan, NOUN, PRCL, INFN]
NUM                           [NUMR, NOUN, nan, ADVB, INFN]
PART      [PRCL, CONJ, ADVB, PRED, nan, INTJ, NPRO, PREP...
PR                      [PREP, ADVB, nan, NOUN, CONJ, VERB]
S         [NOUN, VERB, nan, PREP, NPRO, INTJ, GRND, CONJ...
SPRO        [NPRO, ADJF, CONJ, PRCL, ADJS, nan, NOUN, VERB]
V         [VERB, GRND, INFN, NOUN, PRTS, ADJS, nan, INTJ...
Name: POS_pm, dtype: object

In [ ]:
tags_dict = {'ADJF': ['A', 'ANUM', 'APRO'],
             'ADJS': ['A', 'ANUM', 'APRO'],
             'COMP': ['A'],
             'ADVB': ['ADV', 'ADVPRO'],
             'PRED': ['ADV', 'ADVPRO'],
             'CONJ': ['CONJ'],
             'INTJ': ['INTJ'],
             'NUMR': ['NUM'],
             'PRCL': ['PART'],
             'PREP': ['PR'],
             'NOUN': ['S', 'COM'],
             'NPRO': ['SPRO'],
             'VERB': ['V'],
             'INFN': ['V'],
             'PRTF': ['V'],
             'PRTS': ['V'],
             'GRND': ['V']
             }

def is_morph_match(tag_ms, tag_pm):
    if tag_pm in tags_dict and tag_ms in tags_dict[tag_pm]:
        return True
    else:
        return False

In [ ]:
mono_df_all['morph_match'] = mono_df_all.apply(lambda x: is_morph_match(x.POS_ms, x.POS_pm), axis=1)

In [ ]:
mono_df_all.groupby(by=['morph_match'])['morph_match'].count()

morph_match
False     358832
True     3335928
Name: morph_match, dtype: int64

In [ ]:
mono_df_all.to_csv(f'/content/drive/MyDrive/diplom/mono_df_all_new.csv', quotechar='&')

In [ ]:
print(f'Процент совпадения морфологических тэгов: {3335928/(3335928 + 358832)*100}')

Процент совпадения морфологических тэгов: 90.28808366443288


## Позиция в строке

In [ ]:
verse_data

,clausula,line
0,Я5ж,За о̀пустѐвший сто̀л я вно̀вь садѝлся.
1,Я5м,"Тоску̀я, ду̀мал, ду̀мал о̀б одно̀м."
2,Я5ж,"В твоѐ окно̀ пото̀к черво̀нцев лѝлся,"
3,Я5м,ложѝлся на̀ пол зо̀лоты̀м пятно̀м...
4,Я5ж,"Каза̀лось мнѐ, что ты̀ придѐшь из са̀да"
...,...,...
2440702,Д3м,Нѐ услажда̀ют очѐй;
2440703,Д4д,"Ѝх нищета̀, их терпѐнье безмѐрное"
2440704,Д3м,То̀лько доса̀ду родѝт...
2440705,Д4д,"Что̀ же ты лю̀бишь, дитя̀ маловѐрное,"


In [ ]:
vowels = list('аяоёуюыиэеАЯОЁУЮЫИЭЕ')

def count_vowels(word):
    s = 0
    global vowels
    for letter in word:
        if letter in vowels:
            s += 1
    return s

In [ ]:
def count_syll(line_id, position=-1):
    words = ms.lemmatize(verse_data.loc[line_id]['line'])
    line_segment = ''.join(words[:position])
    return count_vowels(line_segment)

In [ ]:
ms = Mystem()

In [ ]:
count_syll(0, position=-1)

за о̀пустѐвший сто̀л я вно̀вь садѝлся.


11

In [ ]:
mono_df_all['syll_num'] = mono_df_all.apply(lambda x: count_syll(x.line_id, position=x.position), axis=1)
mono_df_all['syll_sum'] = mono_df_all.apply(lambda x: count_syll(x.line_id), axis=1)

In [ ]:
mono_df_all['syll_num_reversed'] = mono_df_all.apply(lambda x: (x.syll_sum - x.syll_num), axis=1)

In [ ]:
mono_df_all

,word,ict,line_id,position,clausula,POS_pm,POS_ms,syll_type,morph_match,syll_num,syll_sum,meter,foot_num,clausula_type,syll_onset,syll_num_reversed
0,за,False,0,0,Я5ж,PREP,PR,open,True,0,11,Я,5,ж,closed,11
1,стол,True,0,4,Я5ж,NOUN,S,closed,True,5,11,Я,5,ж,closed,6
2,я,False,0,6,Я5ж,NPRO,SPRO,open,True,6,11,Я,5,ж,open,5
3,вновь,True,0,8,Я5ж,ADVB,ADV,closed,True,7,11,Я,5,ж,closed,4
4,об,True,1,6,Я5м,PREP,PR,closed,True,7,10,Я,5,м,open,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694755,же,False,2440705,2,Д4д,PRCL,PART,open,True,1,12,Д,4,д,closed,11
3694756,ты,False,2440705,4,Д4д,NPRO,SPRO,open,True,2,12,Д,4,д,closed,10
3694757,где,True,2440706,0,Д3м,ADVB,ADVPRO,open,True,0,7,Д,3,м,closed,7
3694758,же,False,2440706,2,Д3м,PRCL,PART,open,True,1,7,Д,3,м,closed,6


In [ ]:
mono_df_all.to_csv(f'/content/drive/MyDrive/diplom/mono_df_all_new.csv', quotechar='&')

## Тип клаузулы, размер

In [ ]:
def split_claus(claus):
    claus = re.sub(r'[\[\]\*]', '', claus)
    res = re.match(r'([А-Яа-яЁё]+)(\d+)([А-Яа-яЁё]+)', claus)
    if not res:
        return None, None, None
    m, num, cl = res.groups()
    return m, num, cl

In [ ]:
mono_df_all['meter'], mono_df_all['foot_num'], mono_df_all['clausula_type'] = \
    zip(*mono_df_all['clausula'].map(split_claus))

In [ ]:
mono_df_all

,word,ict,line_id,position,clausula,POS_pm,POS_ms,syll_type,morph_match,syll_num,syll_sum,syll_onset,syll_num_reversed,meter,foot_num,clausula_type
0,за,False,0,0,Я5ж,PREP,PR,open,True,0,11,closed,11,Я,5,ж
1,стол,True,0,4,Я5ж,NOUN,S,closed,True,5,11,closed,6,Я,5,ж
2,я,False,0,6,Я5ж,NPRO,SPRO,open,True,6,11,open,5,Я,5,ж
3,вновь,True,0,8,Я5ж,ADVB,ADV,closed,True,7,11,closed,4,Я,5,ж
4,об,True,1,6,Я5м,PREP,PR,closed,True,7,10,open,3,Я,5,м
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694755,же,False,2440705,2,Д4д,PRCL,PART,open,True,1,12,closed,11,Д,4,д
3694756,ты,False,2440705,4,Д4д,NPRO,SPRO,open,True,2,12,closed,10,Д,4,д
3694757,где,True,2440706,0,Д3м,ADVB,ADVPRO,open,True,0,7,closed,7,Д,3,м
3694758,же,False,2440706,2,Д3м,PRCL,PART,open,True,1,7,closed,6,Д,3,м


In [ ]:
mono_df_all.to_csv(f'/content/drive/MyDrive/diplom/mono_df_all_new.csv', quotechar='&')

## Фонетическая структура

In [ ]:
vowels = list('аяоёуюыиэеАЯОЁУЮЫИЭЕ')

def syll_onset(word):
    if word[0] in vowels:
        return 'open'
    else:
        return 'closed'

In [ ]:
mono_df_all['syll_onset'] = mono_df_all.apply(lambda x: syll_onset(x.word), axis=1)

In [ ]:
mono_df_all

,word,ict,line_id,position,clausula,POS_pm,POS_ms,syll_type,morph_match,syll_num,syll_sum,meter,foot_num,clausula_type,syll_onset
0,за,False,0,0,Я5ж,PREP,PR,open,True,0,11,Я,5,ж,closed
1,стол,True,0,4,Я5ж,NOUN,S,closed,True,5,11,Я,5,ж,closed
2,я,False,0,6,Я5ж,NPRO,SPRO,open,True,6,11,Я,5,ж,open
3,вновь,True,0,8,Я5ж,ADVB,ADV,closed,True,7,11,Я,5,ж,closed
4,об,True,1,6,Я5м,PREP,PR,closed,True,7,10,Я,5,м,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694755,же,False,2440705,2,Д4д,PRCL,PART,open,True,1,12,Д,4,д,closed
3694756,ты,False,2440705,4,Д4д,NPRO,SPRO,open,True,2,12,Д,4,д,closed
3694757,где,True,2440706,0,Д3м,ADVB,ADVPRO,open,True,0,7,Д,3,м,closed
3694758,же,False,2440706,2,Д3м,PRCL,PART,open,True,1,7,Д,3,м,closed


## Несколько односложных слов подряд

In [4]:
vowels = list('аяоёуюыиэеАЯОЁУЮЫИЭЕ')

def syll_type(word):
    if word[-1] in vowels:
        return 'open'
    else:
        return 'closed'

def count_vowels(word):
    s = 0
    global vowels
    for letter in word:
        if letter in vowels:
            s += 1
    return s

def syll_onset(word):
    if word[0] in vowels:
        return 'open'
    else:
        return 'closed'

In [5]:
s = 'о̀'
ict = s[1]
print(ict)

̀


In [6]:
def neigh_data(line_id, position):
    line = verse_data.loc[line_id]['line']
    words = ms.lemmatize(line.replace(ict, ''))
    #print(line)
    
    if position > 1:
        left = words[position - 2]
        left_neigh = (count_vowels(left) == 1)
        left_syll = syll_type(left)
    else:
        left_neigh = False
        left_syll = None
    
    try:
        right = words[position + 2]
        right_neigh = (count_vowels(right) == 1)
        right_onset = syll_onset(right)
    except IndexError:
        right_neigh = False
        right_onset = None

    return left_neigh, left_syll, right_neigh, right_onset

In [7]:
neigh_data(2440604, 1)

(False, None, True, 'closed')

In [10]:
mono_df_all['left_neigh'], mono_df_all['left_syll'], mono_df_all['right_neigh'], mono_df_all['right_onset'] = \
    zip(*mono_df_all.progress_apply(lambda x: neigh_data(x.line_id, x.position), axis=1))

100%|██████████| 3694760/3694760 [43:24<00:00, 1418.68it/s]


In [12]:
mono_df_all.to_csv(f'/content/drive/MyDrive/diplom/mono_df_all_new.csv', quotechar='&')

## Итоговая таблица

In [11]:
mono_df_all

,word,ict,line_id,position,clausula,POS_pm,POS_ms,syll_type,morph_match,syll_num,syll_sum,syll_onset,syll_num_reversed,meter,foot_num,clausula_type,left_neigh,left_syll,right_neigh,right_onset
0,за,False,0,0,Я5ж,PREP,PR,open,True,0,11,closed,11,Я,5.0,ж,False,None,False,open
1,стол,True,0,4,Я5ж,NOUN,S,closed,True,5,11,closed,6,Я,5.0,ж,False,closed,True,open
2,я,False,0,6,Я5ж,NPRO,SPRO,open,True,6,11,open,5,Я,5.0,ж,True,closed,True,closed
3,вновь,True,0,8,Я5ж,ADVB,ADV,closed,True,7,11,closed,4,Я,5.0,ж,True,open,False,closed
4,об,True,1,6,Я5м,PREP,PR,closed,True,7,10,open,3,Я,5.0,м,False,closed,False,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694755,же,False,2440705,2,Д4д,PRCL,PART,open,True,1,12,closed,11,Д,4.0,д,True,open,True,closed
3694756,ты,False,2440705,4,Д4д,NPRO,SPRO,open,True,2,12,closed,10,Д,4.0,д,True,open,False,closed
3694757,где,True,2440706,0,Д3м,ADVB,ADVPRO,open,True,0,7,closed,7,Д,3.0,м,False,None,True,closed
3694758,же,False,2440706,2,Д3м,PRCL,PART,open,True,1,7,closed,6,Д,3.0,м,True,open,True,closed
